In [1]:
from utils import InferenceSpec
import sagemaker
import logging
import boto3

In [2]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [3]:
region = sagemaker.Session().boto_region_name
boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client('sagemaker')
logger.info(f'Region = {region}')

Region = us-east-1


#### List model package groups - search by string 

In [4]:
sagemaker_client.list_model_package_groups(NameContains='fraud')['ModelPackageGroupSummaryList']

[{'ModelPackageGroupName': 'fraud-detect-demo',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/fraud-detect-demo',
  'ModelPackageGroupDescription': 'Insurance claim fraud detection',
  'CreationTime': datetime.datetime(2021, 7, 19, 22, 6, 8, 743000, tzinfo=tzlocal()),
  'ModelPackageGroupStatus': 'Completed'}]

### Create a Model Package Group

In [5]:
model_package_group_name = 'BERT-Email-Classifier'
config = {'ModelPackageGroupName': model_package_group_name, 
          'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection'
         }

In [6]:
sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']

[{'ModelPackageGroupName': 'BERT-Email-Classifier',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/bert-email-classifier',
  'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection',
  'CreationTime': datetime.datetime(2021, 8, 4, 21, 5, 3, 705000, tzinfo=tzlocal()),
  'ModelPackageGroupStatus': 'Completed'}]

In [7]:
if not sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']:
    response = sagemaker_client.create_model_package_group(**config)
    logger.info(f'Model Package Group: {model_package_group_name} created!')
    logger.info(f'Response: {response}')
else:
    logger.warning('Model Package Group already exists!')

Model Package Group already exists!


#### Delete a Model Package Group if needed

In [ ]:
model_package_group_name = 'BERT-Email-Classifier'
sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

#### How do I register a trained BERT model to the model registry 

In [33]:
from sagemaker import image_uris
ecr_image = image_uris.retrieve(framework='huggingface',region='us-east-1',version='4.6.1',image_scope='inference',
                    base_framework_version='tensorflow2.4.1', py_version='py37', distribution='ubuntu18.04')
ecr_image

ValueError: Empty SageMaker instance type. For options, see: https://aws.amazon.com/sagemaker/pricing/instance-types

In [ ]:
763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04

1. Define the inference spec 

In [27]:
#ecr_image = '763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-tensorflow-training:2.4.1-transformers4.6.1-gpu-py37-cu110-ubuntu18.04'
inference_spec = InferenceSpec().get_dict(ecr_image=ecr_image, 
                                                    supports_gpu=True,  
                                                    supported_content_types=['text/csv'], 
                                                    supported_mime_types=['text/csv'])
inference_spec

{'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4-transformers4.6-cpu-py37'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.8xlarge',
   'ml.c5.xlarge',
   'ml.c5.2xlarge',
   'ml.c5.4xlarge',
   'ml.c5.9xlarge',
   'ml.c5.18xlarge',
   'ml.p2.xlarge',
   'ml.p2.8xlarge',
   'ml.p2.16xlarge',
   'ml.p3.2xlarge',
   'ml.p3.8xlarge',
   'ml.p3.16xlarge'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge

2. Define model metrics 

In [28]:
model_metrics = {
    "ModelQuality": {
        "Statistics": {
            "ContentType": "application/json",
            "S3Uri": f"s3://foobar/metrics.json",
        }
    },
    "Bias": {
        "Report": {
            "ContentType": "application/json",
            "S3Uri": f"s3://foobar/analysis.json",
        }
    },
}

In [29]:
model_package_input_dict = {"ModelPackageGroupName": model_package_group_name, 
                            "ModelPackageDescription": "BERT classifier", 
                            "ModelApprovalStatus": "PendingManualApproval", 
                            "ModelMetrics": model_metrics}

model_package_input_dict.update(inference_spec)
model_package_input_dict

{'ModelPackageGroupName': 'BERT-Email-Classifier',
 'ModelPackageDescription': 'BERT classifier',
 'ModelApprovalStatus': 'PendingManualApproval',
 'ModelMetrics': {'ModelQuality': {'Statistics': {'ContentType': 'application/json',
    'S3Uri': 's3://foobar/metrics.json'}},
  'Bias': {'Report': {'ContentType': 'application/json',
    'S3Uri': 's3://foobar/analysis.json'}}},
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4-transformers4.6-cpu-py37'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.8xlarge',
   'ml.c5.xlarge',
   'ml.c5.2xlarge',
   'ml.c5.4xlarge',
   'ml.c5.9xlarge',
   'ml.c5.18xlarge',
   'ml.p2.xlarge',
   'm

In [30]:
response = sagemaker_client.create_model_package(**model_package_input_dict)
response

ClientError: An error occurred (ValidationException) when calling the CreateModelPackage operation: Cannot find the requested image: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4-transformers4.6-cpu-py37 with tag: 2.4-transformers4.6-cpu-py37 and digest: null. Please check if your ECR image exists and has proper pull permissions for SageMaker.

#### How do I recreate the BERT model from the model registry and use it to create a real-time inference endpoint

#### How do I create an endpoint using model artifacts in S3 

#### How do I create an endpoint via Model Registry